# 回测引擎调用教程

本教程专注于回测引擎的调用方法，假设您已经有了计算好的因子数据。


## 一、 前置条件

- 已有计算好的因子数据（DataFrame格式，MultiIndex：date, stock_code）
- 已配置好数据管理器（DataManager）

## 🎯 教程目标

本教程将教您如何：
1. 正确导入和初始化回测引擎
2. 使用预计算的因子数据执行回测
3. 配置不同的回测参数
4. 分析和可视化回测结果

## 1.  导入回测引擎

In [11]:
# 基础库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# 添加路径
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath('.')))

# 导入回测引擎
from backtest_engine.engine import BacktestEngine, run_backtest
from backtest_engine.performance import PerformanceAnalyzer
from data_manager.data import DataManager

print("✅ 回测引擎模块导入完成!")

✅ 回测引擎模块导入完成!


## 2. 初始化回测引擎

创建回测引擎实例，配置基本参数。

In [5]:
# 初始化数据管理器
data_manager = DataManager()

# 初始化回测引擎
backtest_engine = BacktestEngine(data_manager)

print("🔧 回测引擎初始化完成!")
print(f"  默认交易费用: {backtest_engine.fee:.3%}")
print(f"  多头方向: {backtest_engine.long_direction}")
print(f"  调仓频率: {backtest_engine.rebalance_freq}")
print(f"  数据管理器: 已连接")

🔧 回测引擎初始化完成
   多头方向: high
   调仓频率: weekly
   交易费用: 0.100%
🔧 回测引擎初始化完成!
  默认交易费用: 0.100%
  多头方向: high
  调仓频率: weekly
  数据管理器: 已连接


## 3. 基本使用方法

假设您已经有了因子数据，以下展示如何调用回测引擎：

In [6]:
# 示例：假设您有以下格式的因子数据
# factor_data: DataFrame with MultiIndex (date, stock_code)

# 方法1: 使用便捷函数（推荐）
def example_backtest_with_factor_data(factor_data):
    """
    使用已有因子数据进行回测的示例
    """
    print("🚀 开始回测...")
    
    # 配置回测参数
    config = {
        'start_date': '2024-01-01',
        'end_date': '2024-02-29',
        'rebalance_freq': 'weekly',  # 周度调仓
        'transaction_cost': 0.0      # 零手续费
    }
    
    # 执行回测
    portfolio_returns, positions = run_backtest(
        factor_data=factor_data,
        data_manager=data_manager,
        **config
    )
    
    print(f"✅ 回测完成!")
    print(f"  组合收益序列长度: {len(portfolio_returns)}")
    if hasattr(positions, '__len__'):
        print(f"  调仓记录: {len(positions)}")
    print(f"  总收益率: {(portfolio_returns + 1).prod() - 1:.4f}")
    
    return portfolio_returns, positions

# 方法2: 直接使用BacktestEngine类
def example_backtest_with_engine(factor_data):
    """
    直接使用BacktestEngine类进行回测
    """
    print("🔧 使用BacktestEngine类...")
    
    # 创建自定义配置的引擎
    custom_engine = BacktestEngine(
        data_manager=data_manager,
        fee=0.001  # 0.1%交易费用
    )
    
    # 注意：这里需要使用实际的回测方法
    # 由于BacktestEngine的接口可能不同，建议使用 run_backtest 函数
    print("💡 建议使用 run_backtest 函数进行回测")
    print("💡 BacktestEngine 类主要用于内部逻辑，推荐使用便捷函数")
    
    return None, None

print("📚 回测方法示例定义完成!")
print("💡 注意：这些是示例函数，需要传入实际的因子数据才能运行")

📚 回测方法示例定义完成!
💡 注意：这些是示例函数，需要传入实际的因子数据才能运行


## 4. 调仓频率配置

回测引擎支持多种调仓频率：

In [ ]:
# 支持的调仓频率
frequencies = {
    'daily': '每日调仓 - 最频繁，计算量大',
    'weekly': '周度调仓 - 推荐，平衡收益和实用性',
    'monthly': '月度调仓 - 较低频率，减少交易成本'
}

# 调仓频率对比示例
def compare_frequencies(factor_data):
    """
    对比不同调仓频率的效果
    """
    results = {}
    
    for freq in ['daily', 'weekly', 'monthly']:
        print(f"\n🔄 测试 {freq} 调仓...")
        
        try:
            returns, positions = run_backtest(
                factor_data=factor_data,
                data_manager=data_manager,
                start_date='2024-01-01',
                end_date='2024-02-29',
                rebalance_freq=freq,
                transaction_cost=0.0
            )
            
            total_return = (returns + 1).prod() - 1
            volatility = returns.std() * np.sqrt(252)
            
            results[freq] = {
                'total_return': total_return,
                'volatility': volatility,
                'rebalance_count': len(positions)
            }
            
            print(f"  收益率: {total_return:.4f}")
            print(f"  波动率: {volatility:.4f}")
            print(f"  调仓次数: {len(positions)}")
            
        except Exception as e:
            print(f"  ❌ {freq} 调仓失败: {e}")
    
    return results

print("\n✅ 频率对比函数定义完成!")

## 5. 性能分析

使用PerformanceAnalyzer分析回测结果：

In [7]:
def analyze_backtest_results(portfolio_returns, factor_data):
    """
    分析回测结果的完整流程
    """
    print("📊 开始性能分析...")
    
    try:
        # 创建性能分析器 - 提供必需的参数
        if isinstance(portfolio_returns, pd.Series):
            portfolio_df = pd.DataFrame({'strategy': portfolio_returns})
        else:
            portfolio_df = portfolio_returns
            
        # 创建简化的master_data
        master_data = pd.DataFrame({
            'date': portfolio_returns.index,
            'next_day_return': portfolio_returns.values
        })
        
        analyzer = PerformanceAnalyzer(
            portfolio_returns=portfolio_df,
            factor_data=factor_data,
            master_data=master_data
        )
        
        # 计算性能指标
        analyzer.calculate_metrics()
        
        # 显示主要指标
        print("\n📈 主要业绩指标:")
        if hasattr(analyzer, 'metrics') and analyzer.metrics is not None:
            for col in analyzer.metrics.columns:
                metrics = analyzer.metrics[col]
                print(f"  {col}:")
                for key, value in metrics.items():
                    if isinstance(value, (int, float)):
                        print(f"    {key}: {value:.4f}")
        
        # 尝试IC分析
        try:
            analyzer.calculate_ic()
            if hasattr(analyzer, 'ic_series') and analyzer.ic_series is not None:
                ic_mean = analyzer.ic_series.mean()
                ic_std = analyzer.ic_series.std()
                icir = ic_mean / ic_std if ic_std > 0 else 0
                
                print(f"\n🎯 IC分析结果:")
                print(f"  IC均值: {ic_mean:.4f}")
                print(f"  ICIR: {icir:.4f}")
        except Exception as ic_error:
            print(f"⚠️ IC分析失败: {ic_error}")
                    
        return {
            'performance_calculated': True,
            'ic_calculated': hasattr(analyzer, 'ic_series')
        }
                    
    except Exception as e:
        print(f"⚠️ 详细分析失败，使用简单指标: {e}")
        
        # 计算基础指标
        total_return = (portfolio_returns + 1).prod() - 1
        volatility = portfolio_returns.std() * np.sqrt(252)
        sharpe = total_return / volatility if volatility > 0 else 0
        
        # 计算最大回撤
        cumulative = (portfolio_returns + 1).cumprod()
        running_max = cumulative.expanding().max()
        drawdown = (cumulative - running_max) / running_max
        max_dd = abs(drawdown.min())
        
        print(f"\n📊 基础指标:")
        print(f"  总收益率: {total_return:.4f}")
        print(f"  年化波动率: {volatility:.4f}")
        print(f"  夏普比率: {sharpe:.4f}")
        print(f"  最大回撤: {max_dd:.4f}")
        
        return {
            'performance_metrics': {
                'total_return': total_return,
                'annualized_volatility': volatility,
                'sharpe_ratio': sharpe,
                'max_drawdown': max_dd
            }
        }

print("✅ 性能分析函数定义完成!")
print("💡 函数已适配新的PerformanceAnalyzer接口")

✅ 性能分析函数定义完成!
💡 函数已适配新的PerformanceAnalyzer接口


## 6. 结果可视化

绘制回测结果图表：

In [14]:
def plot_backtest_results(portfolio_returns):
    """
    绘制回测结果图表
    """
    # 设置中文字体
    plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
    plt.rcParams['axes.unicode_minus'] = False
    
    # 计算累积收益
    cumulative_returns = (portfolio_returns + 1).cumprod()
    
    # 创建图表
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
    
    # 累积收益曲线
    ax1.plot(cumulative_returns.index, cumulative_returns.values, 
             linewidth=2, color='blue', label='策略收益')
    ax1.set_title('📈 策略累积收益曲线', fontsize=14, fontweight='bold')
    ax1.set_ylabel('累积收益倍数', fontsize=12)
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # 添加统计信息
    final_return = cumulative_returns.iloc[-1] - 1
    max_value = cumulative_returns.max()
    min_value = cumulative_returns.min()
    
    ax1.text(0.02, 0.98, 
             f'最终收益: {final_return:.2%}\n最高点: {max_value:.3f}\n最低点: {min_value:.3f}', 
             transform=ax1.transAxes, 
             verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    # 日收益率分布
    ax2.hist(portfolio_returns, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    ax2.set_title('📊 日收益率分布', fontsize=14, fontweight='bold')
    ax2.set_xlabel('日收益率', fontsize=12)
    ax2.set_ylabel('频次', fontsize=12)
    ax2.grid(True, alpha=0.3)
    
    # 添加统计线
    mean_return = portfolio_returns.mean()
    std_return = portfolio_returns.std()
    ax2.axvline(mean_return, color='red', linestyle='--', label=f'均值: {mean_return:.4f}')
    ax2.axvline(mean_return + std_return, color='orange', linestyle='--', alpha=0.7, label='±1σ')
    ax2.axvline(mean_return - std_return, color='orange', linestyle='--', alpha=0.7)
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    print("✅ 图表绘制完成!")

print("✅ 可视化函数定义完成!")

✅ 可视化函数定义完成!


## 7. 最佳实践

### 🎯 关键要点

1. **因子数据格式**: 确保因子数据采用MultiIndex格式（date, stock_code）
2. **调仓频率选择**: 周度调仓通常在收益和实用性之间达到良好平衡
3. **交易费用设置**: 根据实际情况设置合理的交易费用
4. **性能分析**: 不仅关注收益，也要评估风险指标
5. **参数测试**: 测试不同参数组合以优化策略

### ⚠️ 注意事项

- 确保因子数据时间范围与回测期间匹配
- 检查股票代码格式的一致性（建议使用 'XXXXXX.SH' 或 'XXXXXX.SZ' 格式）
- 处理数据缺失值和异常值
- 考虑市场制度变化对回测结果的影响

### 🚀 实际应用

要查看完整的因子计算和回测示例，请参考：
- `factor_library/factor_backtest_template.ipynb` - 完整的因子回测模板
- `factor_library/fundamental/size_factor.py` - 市值因子实际案例

---

**🎉 回测引擎调用教程完成！**

本教程专注于回测引擎的调用方法。如需了解因子计算，请参考 `factor_library` 中的相关教程。

In [13]:
# 🚀 回测引擎使用快速参考指南
print("📚 回测引擎使用快速参考指南")
print("="*50)

# 1. 导入模块
print("1️⃣ 导入必要模块:")
print("   from backtest_engine.engine import run_backtest")
print("   from data_manager.data import DataManager")
print("   from backtest_engine.performance import PerformanceAnalyzer")

# 2. 准备数据
print("\n2️⃣ 准备因子数据:")
print("   factor_data: MultiIndex DataFrame (date, stock_code)")

# 3. 初始化组件
print("\n3️⃣ 初始化数据管理器:")
print("   data_manager = DataManager()")

# 4. 配置参数
print("\n4️⃣ 配置回测参数:")
config_example = {
    'start_date': '2024-01-01',
    'end_date': '2024-02-29',
    'rebalance_freq': 'weekly',
    'transaction_cost': 0.0
}
print(f"   config = {config_example}")

# 5. 执行回测 (示例代码)
print("\n5️⃣ 执行回测:")
print("   returns, positions = run_backtest(")
print("       factor_data=factor_data,")
print("       data_manager=data_manager,")
print("       **config")
print("   )")

# 6. 分析结果 (示例代码)  
print("\n6️⃣ 分析结果:")
print("   analyzer = PerformanceAnalyzer(portfolio_returns, factor_data, master_data)")
print("   results = analyzer.calculate_metrics()")

print("\n✅ 快速参考指南展示完成！")
print("💡 将 factor_data 替换为您的实际因子数据即可运行")
print("💡 完整的工作示例请参考 factor_library 中的实际因子策略")

📚 回测引擎使用快速参考指南
1️⃣ 导入必要模块:
   from backtest_engine.engine import run_backtest
   from data_manager.data import DataManager
   from backtest_engine.performance import PerformanceAnalyzer

2️⃣ 准备因子数据:
   factor_data: MultiIndex DataFrame (date, stock_code)

3️⃣ 初始化数据管理器:
   data_manager = DataManager()

4️⃣ 配置回测参数:
   config = {'start_date': '2024-01-01', 'end_date': '2024-02-29', 'rebalance_freq': 'weekly', 'transaction_cost': 0.0}

5️⃣ 执行回测:
   returns, positions = run_backtest(
       factor_data=factor_data,
       data_manager=data_manager,
       **config
   )

6️⃣ 分析结果:
   analyzer = PerformanceAnalyzer(portfolio_returns, factor_data, master_data)
   results = analyzer.calculate_metrics()

✅ 快速参考指南展示完成！
💡 将 factor_data 替换为您的实际因子数据即可运行
💡 完整的工作示例请参考 factor_library 中的实际因子策略
